In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Display plots inline in the Jupyter Notebook
%matplotlib inline

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import pandas as pd

try:
    df = pd.read_csv('emails.csv', on_bad_lines='skip')
except Exception as e:
    print(f"Failed to load the file: {e}")


Failed to load the file: Error tokenizing data. C error: EOF inside string starting at row 146863


In [6]:
# Display the first few rows of the DataFrame
print(df.head())

# Display the DataFrame's shape (number of rows and columns)
print(f'Shape of DataFrame: {df.shape}')

# Display a concise summary of the DataFrame, including the data types of each column
print(df.info())

NameError: name 'df' is not defined